In [2]:
import cobra
from cobra.io import load_json_model
import numpy as np
import pandas as pd
import ray
#cobra_model = load_json_model("./COBRA_models/GEM_Recon3_thermocurated_redHUMAN_AA.json")



from cobra.io.mat import *
#from itertools import compress



cobra_model = load_matlab_model("./COBRA_models/GEM_Recon3_thermocurated_redHUMAN_AA.mat")

import itertools

def grow_dict(previo, nuevo):  
    """Esto es un .update(overwritte=False)?"""

    not_new_keys = set(previo.keys()).intersection(set(nuevo.keys()))
    new_keys     = set(nuevo.keys()).difference(set(previo.keys()))
    for new in new_keys:
        previo[new] =np.unique(nuevo[new]).tolist() 
    for common in not_new_keys:
        previo[common].extend(np.unique(nuevo[common]).tolist() )
    return previo
# Sacamos los genes de Recon hacia un diccionario
# TODO: convertir est en una horrible dict_comprehension { curse : cursed for c in curseds }
dict_rxn_to_gene = dict()
genes            = [g.id for g in cobra_model.genes]

for id in genes:
    # fmt: off
    rxns             = [r.id for r in cobra_model.genes.get_by_id(id)._reaction] # reacciones asociadas al gen de la iter
    updated_dict  : dict[str, list[str]] = dict(zip(rxns,itertools.repeat([id]))) # zipping con la { reaccion : [id_gen] }
    dict_rxn_to_gene = grow_dict(dict_rxn_to_gene, updated_dict) # merge recursivo
    
def get_data_for_heatmap(explanatory_subgraph_path):
        
    explanatory_subgraph = pd.read_parquet(explanatory_subgraph_path).reset_index(drop=True)

    ray.shutdown()
    ray.init()
    @ray.remote


    def get_genes(node_1, node_2):
        
        if node_2 in dict_rxn_to_gene.keys():
            
            return dict_rxn_to_gene[node_2]
            
        elif node_1 in dict_rxn_to_gene.keys():
            
            return dict_rxn_to_gene[node_1]
            
        else:
            return ['']
        
    g = [] 
    for node_1, node_2 in zip(explanatory_subgraph.node1.tolist(), explanatory_subgraph.node2.tolist()):   
        g.append(get_genes.remote(node_1, node_2 ))

    
    genes = pd.Series(ray.get(g) , name='genes')



    assert genes.shape[0] == explanatory_subgraph.shape[0]


    return pd.concat([genes, explanatory_subgraph], axis=1)




to_R_heatmap_explanatory_subgraph_MASKED_GIN_Fluxes = get_data_for_heatmap("./results/dataframes/new_explanatory_subgraph_MASKED_GIN_Fluxes.parquet.gzip")
to_R_heatmap_explanatory_subgraph_MASKED_GIN_Concentration = get_data_for_heatmap("./results/dataframes/new_explanatory_subgraph_MASKED_GIN_Concentration.parquet.gzip")
to_R_heatmap_explanatory_subgraph_MASKED_GIN_Concen_plus_Fluxes = get_data_for_heatmap("./results/dataframes/new_explanatory_subgraph_MASKED_GIN_Concen_plus_Fluxes.parquet.gzip")

No defined compartments in model model. Compartments will be deduced heuristically using regular expressions.
Using regular expression found the following compartments:c, e, g, i, l, m, n, r, x
2023-02-13 09:35:18,250	INFO worker.py:1518 -- Started a local Ray instance.
2023-02-13 09:35:27,282	INFO worker.py:1518 -- Started a local Ray instance.
2023-02-13 09:35:36,511	INFO worker.py:1518 -- Started a local Ray instance.


In [5]:
to_R_heatmap_explanatory_subgraph_MASKED_GIN_Fluxes.reset_index(drop=True).to_csv(
    "./results/dataframes/new_to_R_heatmap_explanatory_subgraph_MASKED_GIN_Fluxes.csv")
to_R_heatmap_explanatory_subgraph_MASKED_GIN_Concentration.to_csv(
    "./results/dataframes/new_to_R_heatmap_explanatory_subgraph_MASKED_GIN_Concentration.csv")
to_R_heatmap_explanatory_subgraph_MASKED_GIN_Concen_plus_Fluxes.to_csv(
    "./results/dataframes/new_to_R_heatmap_explanatory_subgraph_MASKED_GIN_Concen_plus_Fluxes.csv")

In [4]:
to_R_heatmap_explanatory_subgraph_MASKED_GIN_Fluxes

,genes,node1,node2,0,1,2,3,4,5,6,...,990,991,992,993,994,995,996,997,998,999
0,[],ACGAMtly,acgam_c,0.013827,0.013688,0.013724,0.013774,0.013770,0.013905,0.013735,...,0.013789,0.013654,0.013851,0.013769,0.013740,0.013724,0.013863,0.013768,0.013862,0.013817
1,[],ACGAMtly,acgam_l,0.013807,0.014190,0.013995,0.013689,0.013830,0.013307,0.014122,...,0.014278,0.013590,0.014313,0.014370,0.014078,0.014401,0.013494,0.014307,0.013599,0.013665
2,[10993.1],SERHL,2amac_c,0.013688,0.013786,0.013843,0.013669,0.013705,0.013763,0.013825,...,0.013835,0.013823,0.013924,0.013707,0.013692,0.013924,0.013899,0.014009,0.013825,0.013879
3,[10993.1],SERHL,ser_L_c,0.013548,0.014011,0.013928,0.013760,0.013630,0.013703,0.014012,...,0.014155,0.013747,0.014668,0.014071,0.014016,0.014050,0.013771,0.014038,0.013764,0.013862
4,"[6565.1, 6564.1]",PHELYSALAt,phelysala_e,0.013832,0.013811,0.013868,0.013716,0.013693,0.013727,0.013914,...,0.013830,0.013725,0.013844,0.013923,0.013745,0.013747,0.013810,0.013877,0.013869,0.013735
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
50437,"[30.1, 51.1, 51.2, 3295.1, 1962.1]",ttccoa_x,FAOXC240200x,0.013774,0.013733,0.013785,0.013853,0.013895,0.013869,0.013846,...,0.013829,0.013846,0.013803,0.013796,0.013792,0.013719,0.013707,0.013779,0.013740,0.013779
50438,"[51.1, 8310.1]",ttccoa_x,HMR_3062,0.013718,0.013897,0.013793,0.013708,0.013841,0.013866,0.014008,...,0.013822,0.013815,0.013875,0.013819,0.013888,0.013868,0.013764,0.013677,0.013748,0.013806
50439,"[30.1, 51.1, 3295.1, 1962.1]",ttccoa_x,FAOXC24C22x,0.013813,0.013790,0.013854,0.013822,0.013891,0.013722,0.013924,...,0.013840,0.013796,0.013914,0.013833,0.013883,0.013880,0.013835,0.013892,0.013789,0.013821
50440,[30.1],ttccoa_x,ACACT9p,0.013805,0.013764,0.013747,0.013841,0.013697,0.013938,0.013783,...,0.013802,0.013778,0.013829,0.013718,0.013781,0.013903,0.013882,0.013889,0.013668,0.013780
